In [2]:
import gzip
from collections import defaultdict
import math
import scipy.optimize
from sklearn import svm
import numpy
import string
import random
from sklearn import linear_model
# from surprise import SVD, Reader, Dataset
# from surprise.model_selection import train_test_split
# from surprise import accuracy, BaselineOnly
import pandas as pd
import numpy as np
import warnings
import tensorflow as tf
warnings.filterwarnings("ignore")

In [3]:
df = pd.read_csv('protein_data.csv')
df.drop('Unnamed: 0', axis=1, inplace=True)
df.head()

,Gene A Sequence,Gene B Sequence,Interaction Type,Detection Method
0,MQRLKKFIAKREKGDKGKMKWNSSMDYDSPPSYQDVRRGIFPTAPL...,MSSTLAKIAEIEAEMARTQKNKATAHHLGLLKARLAKLRRELITPK...,Physical association,Two hybrid
1,MQRLKKFIAKREKGDKGKMKWNSSMDYDSPPSYQDVRRGIFPTAPL...,MSSTLAKIAEIEAEMARTQKNKATAHHLGLLKARLAKLRRELITPK...,Physical association,Pull down
2,MQRLKKFIAKREKGDKGKMKWNSSMDYDSPPSYQDVRRGIFPTAPL...,MSSTLAKIAEIEAEMARTQKNKATAHHLGLLKARLAKLRRELITPK...,Physical association,Enzyme linked immunosorbent assay
3,MSSTLAKIAEIEAEMARTQKNKATAHHLGLLKARLAKLRRELITPK...,MSSTLAKIAEIEAEMARTQKNKATAHHLGLLKARLAKLRRELITPK...,Direct interaction,Molecular sieving
4,MTAKMETTFYDDALNASFLPSESGPYGYSNPKILKQSMTLNLADPV...,MSSTLAKIAEIEAEMARTQKNKATAHHLGLLKARLAKLRRELITPK...,Association,Anti bait coimmunoprecipitation


In [19]:
counts = {}
mapping = {}
allData = []
for index, row in df.iterrows():
    aSeq = row['Gene A Sequence']
    bSeq = row['Gene B Sequence']

    counts[aSeq] = counts.get(aSeq, 0) + 1
    counts[bSeq] = counts.get(bSeq, 0) + 1

    if aSeq in mapping:
        mapping[aSeq].add(bSeq)
    else:
        mapping[aSeq] = {bSeq}

    if bSeq in mapping:
        mapping[bSeq].add(aSeq)
    else:
        mapping[bSeq] = {aSeq}
    
    allData.append((aSeq, bSeq, 1))

In [22]:
# splitting into test/train
nTrain = int(len(allData) * 0.9)
nTest = len(allData) - nTrain
interactionsTrain = allData[:nTrain]
interactionsTest = allData[nTrain:]

In [23]:
allGenes = set(df["Gene A Sequence"]).union(set(df["Gene B Sequence"]))

In [24]:
def getAllGenesInteracted(gene):
  return mapping[gene]


In [25]:
def getAllGenesNotInteracted(gene):
  genesInteracted = getAllGenesInteracted(gene)
  return allGenes - genesInteracted

In [26]:
negativeSamples = []

In [30]:
for geneA, geneB, _ in interactionsTrain:

  genesNotInteractedA = getAllGenesNotInteracted(geneA)
  genesNotInteractedB = getAllGenesNotInteracted(geneB)
  
  randomGeneA = random.choice(list(genesNotInteractedA))
  randomGeneB = random.choice(list(genesNotInteractedB))

  negativeSampleA = (geneA, randomGeneA, 0)
  negativeSampleB = (geneB, randomGeneB, 0)
  
  negativeSamples.append(negativeSampleA)
  negativeSamples.append(negativeSampleB)


In [ ]:
negativeSamples[:10]

[('MQRLKKFIAKREKGDKGKMKWNSSMDYDSPPSYQDVRRGIFPTAPLFGMEDDMMEFTPSLGIQTLKLQYKCVVNINAINPFRDFREAISAMQFWEADYSGYIGKKPFYRAIILHTARQLKTSNPGILDRGVVEYHATTQGRALVFHSLGPSPSMMFVPETFTREWNILTNKGTINVKIWLGETDTLSELEPILNPVNFRDDREMIEGAAIMGLEIKKQKDNTWLISKSH',
  'MKYILVTGGVISGIGKGIIASSVGTILKSCGLHVTSIKIDPYINIDAGTFSPYEHGEVFVLDDGGEVDLDLGNYERFLDIRLTKDNNLTTGKIYQYVINKERKGDYLGKTVQVVPHITDAIQEWVMRQALIPVDEDGLEPQVCVIELGGTVGDIESMPFIEAFRQFQFKVKRENFCNIHVSLVPQPSSTGEQKTKPTQNSVRELRGLGLSPDLVVCRCSNPLDTSVKEKISMFCHVEPEQVICVHDVSSIYRVPLLLEEQGVVDYFLRRLDLPIERQPRKMLMKWKEMADRYDRLLETCSIALVGKYTKFSDSYASVIKALEHSALAINHKLEIKYIDSADLEPITSQEEPVRYHEAWQKLCSAHGVLVPGGFGVRGTEGKIQAIAWARNQKKPFLGVCLGMQLAVVEFSRNVLGWQDANSTEFDPTTSHPVVVDMPEHNPGQMGGTMRLGKRRTLFQTKNSVMRKLYGDADYLEERHRHRFEVNPVWKKCLEEQGLKFVGQDVEGERMEIVELEDHPFFVGVQYHPEFLSRPIKPSPPYFGLLLASVGRLSHYLQKGCRLSPRDTYSDRSGSSSPDSEITELKFPSINHD',
  0),
 ('MSSTLAKIAEIEAEMARTQKNKATAHHLGLLKARLAKLRRELITPKGGGGGGPGEGFDVAKTGDARIGFVGFPSVGKSTLLSNLAGVYSEVAAYEFTTLTTVPGVIRYKGAKIQLLDLPGIIEGAKDGKGRGRQVIAVARTCNLILIVLDVLKPLGHKK

In [ ]:
interactionsTrain = interactionsTrain + negativeSamples